<h1>SI 564 Final Project: Trails in U.S. National Parks</h1>
<p>Haley Johnson</p>

<p>Solution key to homework assignment. I used starter code from <a href = "https://gist.github.com/mlhess/be7841fda8baba5645f6aad807b50083">here</a> to connect to SQL</p>

In [1]:
import os
import pymysql
import pandas as pd

In [2]:
con = pymysql.connect(host='34.134.16.183', port=14192,user='haleyej-ro',password=p,database='nat_parks',cursorclass=pymysql.cursors.DictCursor)  
cur = con.cursor()

<h4>Question 1</h4>
<p>What national park has the longest trail length, on average?</p>

In [3]:
cur.execute('''SELECT p.park, round(avg(t.length), 2) as average_length
               FROM trails t
               JOIN parks p 
               ON t.park_id = p.id
               GROUP BY p.park
               ORDER BY avg(t.length) DESC
               LIMIT 1;''')
for row in cur:
    print(row['park'], row['average_length'])

Kings Canyon 38162.46


<h4>Question 2</h4>
<p>What is the average elevation gain of trails that are in national parks where ‘Mountain,’ ‘Mt.’ or any variation of Mountain is in the park's name? </p>

In [4]:
cur.execute('''SELECT p.park, round(avg(t.elevation_gain), 2) as average_elevation_gain
               FROM trails t
               JOIN parks p 
               ON t.park_id = p.id
               WHERE p.park LIKE '%Mountain%'
               GROUP BY p.park
               ORDER BY avg(t.elevation_gain) DESC;''')
for row in cur:
    print(row['park'], row['average_elevation_gain'])

Great Smoky Mountains 848.22
Guadalupe Mountains 735.5
Rocky Mountain 590.13


<h4>Question 3</h4>
<p>What is the most popular trail in each national park?</p>

In [5]:
cur.execute('''SELECT p.park, t.name''')

OperationalError: (1109, "Unknown table 'p' in field list")

<h4>Question 4</h4>
<p>Alaska has the most national parks out of any state. What are the 5 most popular trails in national parks located in Alaska? </p>

In [10]:
cur.execute('''SELECT p.park, t.name, t.popularity
               FROM trails t
               JOIN parks p 
               ON t.park_id = p.id
               JOIN states s
               ON t.state_id = s.id
               WHERE s.state_name = 'Alaska'
               ORDER BY t.popularity DESC
               LIMIT 5;''')
for row in cur:
    print(f"Trail: {row['name']}\nPark: {row['park']}\nPopularity Rating {row['popularity']}\n")

Trail: Harding Ice Field Trail
Park: Kenai Fjords
Popularity Rating 24.8931

Trail: Mount Healy Overlook Trail
Park: Denali
Popularity Rating 18.0311

Trail: Exit Glacier Trail
Park: Kenai Fjords
Popularity Rating 17.7821

Trail: Horseshoe Lake Trail
Park: Denali
Popularity Rating 16.2674

Trail: Triple Lakes Trail
Park: Denali
Popularity Rating 12.5935



<h4>Question 5</h4>
<p>How many trails are in Yellowstone National Park? </p>

In [11]:
cur.execute('''SELECT count(1) AS number_of_trails
               FROM trails t
               JOIN parks p 
               ON t.park_id = p.id
               WHERE p.park = 'Yellowstone';''')


for row in cur:
    print(f"Number of trails in Yellowstone National Park: {row['number_of_trails']}")

Number of trails in Yellowstone National Park: 228


<h4>Question 6</h4>
<p>Return a list of 5 random trails that are in the contiguous U.S. and have less than 500 feet in elevation gain. </p>

In [15]:
cur.execute('''SELECT p.park, t.name, t.elevation_gain
               FROM trails t
               JOIN parks p 
               ON t.park_id = p.id
               JOIN states s 
               ON t.state_id = s.id
               WHERE s.state_name NOT IN ('Alaska', 'Hawaii')
                     AND t.elevation_gain < (500/3)
               ORDER BY RAND()
               LIMIT 5;''')


for row in cur:
    print(f"Trail: {row['name']}\nPark: {row['park']}\nElevation Gain In Yards: {row['elevation_gain']}\n")

Trail: Gould Mine
Park: Saguaro
Elevation Gain In Yards: 115.824

Trail: Many Pools Trail
Park: Zion
Elevation Gain In Yards: 165.8112

Trail: Point Sublime Trail
Park: Yellowstone
Elevation Gain In Yards: 103.9368

Trail: Fountain Paint Pot Trail
Park: Yellowstone
Elevation Gain In Yards: 6.7056

Trail: The Beehive Loop Trail
Park: Acadia
Elevation Gain In Yards: 148.7424



<h4>Question 7</h4>
<p>Which national park has the most trails with the maximum difficulty rating?  </p>

In [34]:
cur.execute('''SELECT p.park, count(1) AS num_parks
               FROM trails t 
               JOIN parks p 
               ON t.park_id = p.id
               WHERE t.difficulty_rating = 
                (SELECT max(difficulty_rating) FROM trails t)
               GROUP BY p.park
               ORDER BY 2 DESC
               LIMIT 1;''')


for row in cur:
    print(f"{row['park']} National Park has {row['num_parks']} parks with the maximum difficulty rating")

Yosemite National Park has 25 parks with the maximum difficulty rating


<h4>Question 8</h4>
<p>What trails have high average visitor ratings (4.5 or higher) but have the lowerst visitor usage rating</p>

In [42]:
cur.execute('''SELECT t.name, p.park, t.avg_rating, t.visitor_usage
                FROM trails t 
                JOIN parks p
                ON t.park_id = p.id
                WHERE t.avg_rating >= 4.5 
                      AND t.visitor_usage = 
                      (SELECT min(visitor_usage) FROM trails)
                ORDER BY rand()
                LIMIT 20;''')

for row in cur:
    print(f'''{row['name']} in {row['park']}\nAverage Rating: {row['avg_rating']}\n''')

Firebrand Trail to Ole Creek in Glacier
Average Rating: 4.5

Lawn Lake Trail in Rocky Mountain
Average Rating: 4.5

West Creek Falls and Fox Creek Falls Trail in Rocky Mountain
Average Rating: 4.5

Appalachian Trail: Clingman Dome to Cades Cove in Great Smoky Mountains
Average Rating: 4.5

Lake Solitude via Lake Solitude and Cascade Canyon Trail in Grand Teton
Average Rating: 5.0

Baskins Creek Trail and Baskins Creek Falls in Great Smoky Mountains
Average Rating: 4.5

Hole-in-the-Wall Trail in Olympic
Average Rating: 4.5

Black Bay Beaver Pond Trail in Voyageurs
Average Rating: 5.0

Bumpass Hell from Kohm Yah-mah-nee visitor center in Lassen Volcanic
Average Rating: 5.0

Lobo Canyon Trail in Channel Islands
Average Rating: 4.5

Green River Great Overlook in Canyonlands
Average Rating: 4.5

Piegan Pass Trail in Glacier
Average Rating: 4.5

Little Rock Lake via Ute Trail and Mount Ida Trail in Rocky Mountain
Average Rating: 4.5

Pleasant Creek Trail in Capitol Reef
Average Rating: 4.5



<h4>Question 9</h4>
<p>What are the 5 most unpopular trails in the 5 most popular national parks</p>
</br>
<p><b>Note:</b> there are many ways to interpret popularity. Any solution that uses fairly logical criteria to operationalize popularity and appropriately justifies their answers should receive full credit for that component of the problem.</p>

In [44]:
cur.execute('''SELECT t.name, t.


               (SELECT t.park_id
                FROM trails t
                GROUP BY t.park_id
                ORDER BY AVG(t.popularity) DESC
                LIMIT 5)''')

{'park_id': 0}
{'park_id': 48}
{'park_id': 59}
{'park_id': 47}
{'park_id': 42}
